# flowField class for iterative solution

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from flowField import *
from flowFieldWavy import *
import h5py
import numpy as np
import pdb
import cProfile
import time
from pseudo import *
from scipy.linalg import norm, svd
from scipy.sparse.linalg import gmres, LinearOperator
from lgmresCustom import lgmres

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%run test_flowFieldWavy.py

In [ ]:
def vecnorm2(vec,N):
    m = vec.size//N
    w = clencurt(N).reshape((1,N))
    W = np.tile(w,(1,m))
    tempVec = vec.reshape(vec.size,1)
    tempVecConj = tempVec.conj().T
    
    vecNorm = np.sqrt(np.abs( np.dot( tempVecConj*W, tempVec ) ))
    return vecNorm[0,0]

#flowDict = tempDict
finDifEps =1.0e-7
def residual(xF,**kwargs):
    #return (xF.slice(nd=[0,1,2]).residuals(pField=xF.getScalar(nd=3))[0]).appendField(xF.slice(nd=[0,1,2]).div())
    #xTemp = xF.slice(L=xF.nx//2+2,M=xF.nz//2+2)
    #xTemp = xF.slice(L=xF.nx//2+2)
    xTemp = xF
    resid = (xTemp.slice(nd=[0,1,2]).residuals(pField=xTemp.getScalar(nd=3),**kwargs)[0]).appendField(xTemp.slice(nd=[0,1,2]).div())
    #return resid.slice(L=xF.nx//2,M=xF.nz//2)
    return resid # .slice(L=xF.nx//2)


def jacobian(x0,r0,arr):
    xFF = weighted2ff(flowDict=x0.flowDict, arr=arr)
    #vf = finDifEps*xFF.slice(nd=[0,1,2]) + x0.slice(nd=[0,1,2])
    #pf = finDifEps*xFF.getScalar(nd=3) + x0.slice(nd=3)
    #residual = vf.residuals(pField=pf)[0]
    #residual= residual.appendField(vf.div())
    
    matVecProd = (residual(x0+finDifEps*xFF)-r0)/finDifEps
    return matVecProd.weighted()



searchArr = np.append(np.arange(0.,1.5),np.array([2.0]))
searchArr = np.append(-searchArr[::-1],searchArr[1:])
def lineSearch(x0,arr):
    delFF = (weighted2ff(flowDict=x0.flowDict,arr=arr).view4d())
    resArr = np.zeros(searchArr.size)
    k = 0
    for step in searchArr:
        resArr[k] = residual(x0 + step*delFF).norm()
        k +=1
    # print(resArr)
    optimalStep = searchArr[resArr.argmin()]
    print('Optimal factor from line search is ', optimalStep)
    return (x0 + optimalStep* delFF)

searchArrFine = np.arange(-0.1,0.11,0.01)
def lineSearchFine(x0,arr):
    delFF = (weighted2ff(flowDict=x0.flowDict,arr=arr).view4d())
    resArr = np.zeros(searchArrFine.size)
    k = 0
    for step in searchArrFine:
        resArr[k] = residual(x0 + step*delFF).norm()
            
    print(resArr)
    print(searchArrFine)
    optimalStep = searchArrFine[resArr.argmin()]
    print('Optimal factor from line search is ', optimalStep)
    return (x0 + optimalStep* delFF)
    
        

In [ ]:
lda = 25.

def solvePressure(vf):
    # To impose BCs, I need to evaluate the diffusion and convection terms at the wall (along y)
    convTerm  = vf.convNL()
    convTermY = convTerm.getScalar(nd=1)
    diffTermY = vf.getScalar(nd=1).laplacian()/vf.flowDict['Re']
    tempTerm = -convTermY + diffTermY + lda*vf.getScalar(nd=1)
    
    RHSterm = -convTerm.div()
    # Replacing all wall-entries with the appropriate BCs:
    RHSterm[:,:,:,:,[0,-1]] = tempTerm[:,:,:,:,[0,-1]]
    RHSterm = RHSterm.copyArray().reshape(vf.nx, RHSterm.size//vf.nx)

    pArr = np.zeros((vf.nx,n),dtype=np.complex)
    for lp in range(vf.nx):
        pArr[lp] = np.dot(invDelBC[lp],RHSterm[lp])

    _pf = diffTermY.zero()
    _pf.view1d()[:] = pArr.reshape(pArr.size)
    return _pf.view4d()

def residualV2(vf):
    _pf = solvePressure(vf)
    return residual(vf.appendField(_pf)).slice(nd=[0,1,2])

def jacobianV2(vf0,r0,arr):
    vfCorr = weighted2ff(flowDict=vf0.flowDict, arr=arr)
    matVecProd = (residualV2(vf0+finDifEps*vfCorr)-r0)/finDifEps
    return matVecProd.weighted()

def divFree(vf):
    gz = vf.flowDict['eps']*vf.flowDict['beta']
    vf[:,:,1:,1] += 1.j*gz*vf[:,:,:-1,2]
    vf[:,:,:-1,1]-= 1.j*gz*vf[:,:,1: ,2]
    return 

def jacobianV3(vf0,r0,arr):
    vfCorrect = weighted2ff(flowDict=vf0.flowDict, arr=arr)
    matVecProd = (residualV3(vf0+finDifEps*vfCorrect)-r0)/finDifEps
    return matVecProd.weighted()

def residualV3(vf):
    convTerm = vf.convNL()
    diffTerm = vf.laplacian()/vf.flowDict['Re']
    
    convTermY = convTerm.getScalar(nd=1)
    diffTermY = vf.getScalar(nd=1).laplacian()/vf.flowDict['Re']
    tempTerm = -convTermY + diffTermY + lda*vf.getScalar(nd=1)
    
    RHSterm = -convTerm.div()
    # Replacing all wall-entries with the appropriate BCs:
    RHSterm[:,:,:,:,[0,-1]] = tempTerm[:,:,:,:,[0,-1]]
    RHSterm = RHSterm.copyArray().reshape(vf.nx, RHSterm.size//vf.nx)

    pArr = np.zeros((vf.nx,n),dtype=np.complex)
    for lp in range(vf.nx):
        pArr[lp] = np.dot(invDelBC[lp],RHSterm[lp])

    _pf = diffTermY.zero()
    _pf.view1d()[:] = pArr.reshape(pArr.size)
    
    # Residual = d_t(u,v,w), without BCs imposed
    residual = -_pf.view4d().grad() + diffTerm - convTerm
    
    if True:
        return residual
    # BCs on x and z momentum equations are the velocity BCs:
    residual[:,:,:,0,[0,-1]] = -vf[:,:,:,0,[0,-1]]     # set d_t(u) = -u for all modes
    residual[:,vf.nx//2,vf.nz//2,0,0 ] += 1.           # For (0,0) mode, set d_t(u) = +/-1 - u
    residual[:,vf.nx//2,vf.nz//2,0,-1] += -1.
    
    residual[:,:,:,2,[0,-1]] -= -vf[:,:,:,2,[0,-1]]     # set d_t(w) = -w for all modes
    
    # BC on y-momentum is the divergence-free condition. Set the residual as -div(u,v,w) at y = +/- 1
    vfDiv = vf.div()
    residual[:,:,:,1,[0,-1]] -= -vfDiv[:,:,:,0,[0,-1]]
    
    return residual
    

In [ ]:
vF = h52ff("eq1.h5")
pF = h52ff("eq1_pressure.h5",pres=True)
x00 = vF.appendField(pF)



In [ ]:
res = pF.laplacian() + vF.convNL().div()
vecnorm2(res[:,:,:,:,1:N-1], res.N-2), res.norm()/pF.laplacian().norm(), residual(x00).norm()

In [ ]:
res = pC.laplacian() + vF.convNL().div()
res.norm(), vecnorm2(res[:,:,:,:,1:N-1], res.N-2)


In [ ]:
pCNorm = np.zeros(pC.nx)
for l in range(pC.nx):
    pCNorm[l] = vecnorm2( np.dot(DelBC[l],pC[0,l].reshape(pC.size//pC.nx)) - RHSterm[l],  pC.N)

print(np.sum(pCNorm))


In [ ]:
yRes = pC.ddy() - vF.getScalar(nd=1).laplacian()/vF.flowDict['Re']+ vF.convNL().getScalar(nd=1)


In [ ]:
%timeit vF.convNL()
%timeit residualV3(vF)
%timeit residual(x00)
%timeit x00.slice(nd=[0,1,2]).residuals(pField=x00.getScalar(nd=3))
xTemp = x00.slice(L=10,M=10)
%timeit residual(xTemp)

In [ ]:
r = residualV3(vF)
r[:,:,:,:,[0,-1]] = 0.
r.div().norm()

In [ ]:
vecnorm2(r.div()[:,:,:,:,1:-1], vF.N-2)

In [ ]:
norm(vF.div()[:,:,:,:,[0,-1]])

In [ ]:
#(solvePressure(vF)-pF).norm()
pC = solvePressure(vF)

pDiffNorm = np.zeros(vF.nx)

for l in range(vF.nx):
    pDiffNorm[l] = vecnorm2( np.dot( DelBC[l], pF[0,l].reshape(pF.size//pF.nx) ) 
                            - np.dot(DelBC[l], pC[0,l].reshape(pC.size//pC.nx)), pF.N)

#for l in range(vF.nx):
    #pDiffNorm[l] = vecnorm2( np.dot( DelNoBC[l], (pF[0,l]-pC[0,l]).reshape(pF.size//pF.nx) )  , pF.N)
    
    
np.sum(pDiffNorm), pF.flowDict['eps']



In [ ]:
vf = vF.copy()
# To impose BCs, I need to evaluate the diffusion and convection terms at the wall (along y)
convTerm  = vf.convNL()
convTermY = convTerm.getScalar(nd=1)
diffTermY = vf.getScalar(nd=1).laplacian()/vf.flowDict['Re']
#tempTerm = -convTermY + diffTermY + lda*vf.getScalar(nd=1)
tempTerm = diffTermY

RHSterm = -convTerm.div()
# Replacing all wall-entries with the appropriate BCs:
RHSterm[:,:,:,:,[0,-1]] = tempTerm[:,:,:,:,[0,-1]]
RHSterm = RHSterm.copyArray().reshape(vf.nx, RHSterm.size//vf.nx)

pCNorm = np.zeros(vF.nx)
for l in range(vF.nx):
    pCNorm[l] = vecnorm2( np.dot( DelBC[l], pC[0,l].reshape(pF.size//pF.nx) ) - RHSterm[l] , pF.N)

print(pCNorm)
print(np.sum(pCNorm))


In [ ]:
laplpF = pF.lapl()

## GMRES using pressure solver

In [ ]:
vf0 = vF.copy()
vf0.flowDict['eps'] = 0.00
#divFree(vf0)

startTime = time.time()
r0 = residualV3(vf0)
r0norm = r0.norm()
print('$||r_0|| $',r0norm)
for iter in range(1):
    if r0norm < 1.0e-15: print('Norm below tolerance, exiting....',r0norm); break
    jcbn = lambda arr: jacobianV3(vf0,r0,arr)
    A = LinearOperator((vf0.size,vf0.size), matvec=jcbn,dtype=np.complex)
    delX,W,V,H,flag = lgmres(A,-r0.weighted(),tol=1.0e-3,maxiter=1,inner_m=5,outer_k=0)
    if flag: print('LGMRES did not converge...................................')
    print('||Ax-b||:',norm(A.matvec(delX)+r0.weighted(),2))
    print('||Ax-b||/||b||:',norm(A.matvec(delX)+r0.weighted(),2)/norm(r0.weighted(),2))
    break
    vf0 += weighted2ff(arr=delX,flowDict=vf0.flowDict)
    r0 = residualV2(vf0)
    print('New and old norms respectively:',r0.norm(), r0norm)
    print('****************************')
    r0norm = r0.norm()
    
print('Done........................................')    
print('Time elapsed:',int(time.time()-startTime))

In [ ]:
ffList = []
for k in range(5):
    vField = weighted2ff(arr=V[k], flowDict=vf0.flowDict)
    ffList.append(vField)


In [ ]:
vF.div().norm()
res = residualV3(vf0)
res.div().norm()/res.norm(), ffList[0].div().norm()/ffList[0].norm(), vf0.div().norm()/vf0.norm()
vecnorm2(res.div()[:,:,:,:,1:-1], res.N-2)
vecnorm2(ffList[3].div()[:,:,:,:,1:-1], res.N-2)
#((res/res.norm()*ffList[0].norm()) + ffList[0]).norm()
#ffList[0].div().norm()

## Pressure Poisson solver: Inverse of Del operator

In [ ]:
# Creating Del_{l} matrices (these don't include BCs on pressure)
# Different Del_l matrices only differ by a -l^2 a^2 on the diagonal,
#  so creating a base matrix that can generate them all later

L = vf.nx//2; M= vf.nz//2; N=vf.N; n= vf.nz*vf.N
ms = 1    # m_s in my equations, functionality to be added later
DelBase = np.matrix( np.zeros((n,n+4*ms*N),dtype=np.complex)  )
b = vf.flowDict['beta']; a = vf.flowDict['alpha']; eps = vf.flowDict['eps']
gz = eps*b
Ns = 2*ms*N           # Makes it easier to define matrix (because some modes are absent)

for mp in range(vf.nz):
    m = mp-M
    # First, diagonal blocks corresponding to (m,m): (-m^2 b^2)I + (1+2g_z^2)D^2
    DelBase[ N*mp: N*(mp+1), Ns+N*mp: Ns+N*mp+N ] = -(m*b)**2 * np.identity(N) + (1.+2.*gz**2) * vf.D2
    
    DelBase[ N*mp: N*(mp+1), Ns+N*(mp-2*ms): Ns+N*(mp-2*ms)+N ] = -gz**2 * vf.D2
    DelBase[ N*mp: N*(mp+1), Ns+N*(mp+2*ms): Ns+N*(mp+2*ms)+N ] = -gz**2 * vf.D2
    
    DelBase[ N*mp: N*(mp+1), Ns+N*(mp-ms): Ns+N*(mp-ms)+N ] = ( 2*m-ms)*gz*b * vf.D
    DelBase[ N*mp: N*(mp+1), Ns+N*(mp+ms): Ns+N*(mp+ms)+N ] = (-2*m-ms)*gz*b * vf.D

DelBase = DelBase[:,Ns:-Ns]

DelNoBC = np.zeros((vf.nx,n,n), dtype=np.complex)
for lp in range(vf.nx):
    l = lp-L
    DelNoBC[lp] += DelBase
    for mp in range(vf.nz):
        DelNoBC[lp, N*mp: N*(mp+1), N*mp: N*(mp+1) ] += -(l*a)**2 * np.identity(N)

D = vf.D

# Applying Neumann BCs on Del:
DelBC = DelNoBC.copy()
for lp in range(vf.nx):
    DelBC[lp, N*mp    , :  ] = 0.      # Setting equations at wall to 0.*p
    DelBC[lp, N*(mp+1)-1, :  ] = 0.        
    for mp in range(vf.nz):
        DelBC[lp, N*mp    , N*mp: N*(mp+1) ] = D[0]           # Changing equations at wall to D*p_{l,m}
        DelBC[lp, N*(mp+1)-1, N*mp: N*(mp+1) ] = D[-1]
        # The RHS for D*p(y=wall) has to be set separately

# DelBC is not dependent on the velocity field at all
#   So, if I invert this once, that should be enough
invDelBC = np.zeros(DelBC.shape, dtype=np.complex)
for lp in range(vf.nx):
    invDelBC[lp] = np.linalg.pinv(DelBC[lp])

### Testing the DelNoBC and DelBC matrices with known pressure fields

In [ ]:
pExArr = pF.view4d().copyArray()
pExArr = pExArr.reshape(vf.nx, pExArr.size//vf.nx)

In [ ]:
pNorm = np.zeros(vf.nx)
for l in range(vf.nx):
    pNorm[l] = vecnorm2( np.dot(DelNoBC[l],pExArr[l]) - RHSterm[l], vf.N   )

pNorm

In [ ]:
(pfCustom.laplacian()+convTerm.div()).norm()

In [ ]:
pLaplArr = pF.laplacian().copyArray().reshape(pF.nx, pF.size//pF.nx)
pFArr = pF.view4d().copyArray().reshape(pF.nx, pF.size//pF.nx)
for l in range(pF.nx):
    pNorm[l] = vecnorm2( pLaplArr[l] - np.dot(DelNoBC[l],pFArr[l]), pF.N )

pNorm


In [ ]:
vf0.div().norm()

## Using numpy arrays for gmres instead of flowFieldWavy
If I do that, I can use scipy's gmres and other routines. The plan is this- supply weighted np.ndarray copies of flowFieldWavy instances, and when the Jacobian matvec needs to be computed, make a flowField instance (actually, just assign to an existing instance) out of the array. 

In [ ]:
%timeit residual(x0,fft=True)
%timeit residual(x0,fft=False)

In [ ]:
#x0 = x00.slice(L=7,M=7)
x0 = x0.slice(L=15,M=15)
x0.flowDict['eps'] = 0.000

#x0 = x00.copy()
#x0[:]=0.; x0[0,x0.nx//2,x0.nz//2,0] = 1.-x0.y**2
startTime = time.time()
r0 = residual(x0)
r0norm = r0.norm()
print('$||r_0|| $',r0norm)
for iter in range(1):
    if r0norm < 1.0e-15: print('Norm below tolerance, exiting....',r0norm); break
    jcbn = lambda arr: jacobian(x0,r0,arr)
    A = LinearOperator((x0.size,x0.size), matvec=jcbn,dtype=np.complex)
    delX,W,V,H,flag = lgmres(A,-r0.weighted(),tol=1.0e-3,maxiter=1,inner_m=100,outer_k=0)
    if flag: print('LGMRES did not converge...................................')
    print('||Ax-b||/||b||:',norm(A.matvec(delX)+r0.weighted(),2)/norm(r0.weighted(),2))
    #break
    #print('||del X||:',norm(delX,2))
    x0[:] = lineSearch(x0,delX)
    r0 = residual(x0)
    print('New and old norms respectively:',r0.norm(), r0norm)
    print('****************************')
    r0norm = r0.norm()
    
print('Done........................................')    
print('Time elapsed:',int(time.time()-startTime))

In [ ]:
# Verify that V[:-1] == W
print('Is V[:-1] the same as W?:',(V[:-1]-W ==0).all())

# Verify Orthonormality of vectors in V and W:
testTol = 1.0e-12
dotMat = np.dot(W, W.T.conj())
norm2Arr = dotMat.diagonal().copy()
np.fill_diagonal(dotMat, 0.)
dotSum = np.sum( np.sum( np.abs(dotMat), axis=0  ), axis=0)/(W.shape[0]**2)
print('Is the basis orthogonal?',dotSum<=testTol)
print('Are basis vectors unit vectors?:',(np.abs(norm2Arr-1.) <= testTol).all())

In [ ]:
# Save flowFieldRiblet object to binary file whenever happy with the residual norm
x00 = x0.copy()
np.save('eq1Wavy1.npy',x00)

In [ ]:
xTemp = np.load('eq1Custom.npy')



In [ ]:
x00[:] = xTemp
residual(x00).norm()

In [ ]:
# Writing to hdf5 format to run in channel flow
#    Simplest way to go about this would be to load a h5 file and then change one of its entries.

# First, ordering my array in the way the h5 files are written
# They need to be written in physical, and ordered as component, x, y, z
# I'll simply reverse whatever I did in the h52ff function of flowFieldWavy module

x = x00.slice(L=16,M=16).copy()
x[0,16,16,0] -= x.y
nx = x.nx; nz=x.nz; N= x.N
# Split up components first, and then put them on axis 0:
v1SpecArr = x.getScalar().copyArray().reshape((1,nx,nz,N))
v2SpecArr = x.getScalar(nd=1).copyArray().reshape((1,nx,nz,N))
v3SpecArr = x.getScalar(nd=2).copyArray().reshape((1,nx,nz,N))
vSpecArr = np.concatenate( (v1SpecArr, v2SpecArr, v3SpecArr), axis=0 )

# Do ifft to get to physical
# numpy likes it when the modes go 0,1,..,N-1, -N, -N+1,.., -1: There's no +N
# Dropping the largest positive modes
vSpecArr = vSpecArr[:,:-1,:-1] ; nx -= 1; nz -= 1;
vPhysArr = np.real(  np.fft.ifftn(  np.fft.ifftshift(vSpecArr, axes=[1,2]), axes=[1,2] ) * nx*nz  )

# Finally, reordering from x,z,y to x,y,z:
vT = np.zeros((3,nx,N,nz))
for k in range(nz):
    vT[:,:,:,k] = vPhysArr[:,:,k]

# vT is ready to go into the h5 file. 
# Reading an h5 file so I don't have to define too many things:
h5File = h5py.File('eq1Custom.h5','r+')
# h5File has all the data stored as the appropriate structure
# I only need to modify the velocity vector before saving to a different file
uh5 = h5File['data']['u']
uh5[...] = vT
h5File.close()

hFile = h5py.File('eq1Custom.h5','r')
uT = np.array(hFile['data']['u'])
norm( (uT-vT).reshape(vT.size)   )

## Verifying H and W are consistent with A 
* Ax = b

Since numpy likes populating the last dimension first, the basis vectors are stored as rows of V and W instead of columns 
* H is the Hessenberg matrix, and  A V.T = W.T H
* A is of size m,m
* V is of size m,n
* W is of size m,n-1
* H is of size n-1,n

The k^th column of H gives the components of the A*V[:,k] 



In [ ]:
# First, verifying that V[:,:-1] and W are the same matrix


Q = W.conj().T
Qm1 = Q[:,:-1]
tmpMat = A.matvec(Q)
norm( A.matvec(Q[:,0]) - np.dot(Q,H)[:,0], 2)

## Locally constrained Hookstep: Part 1

 Channelflow.org uses the following convention for GMRES/Hookstep of Ax=b:
*    Q_n-1, Q_n: Arnoldi basis vector spaces
*    H_n : Hessenberg matrix such that A*Q_n = Q_n-1 * H
*    U D V* = H_n, the SVD of H_n
*    \hat{b} = U* b
*    s = Q_n* x = Components of the correction 'x' along the basis Q_n
*       's' is the vector that needs to be computed
*    \hat{s} = V* s
*    \hat{s} is given as 
*        \hat{s}_i = (\hat{b}_i d_i)/(d_i^2 + \mu)


In [ ]:
delta = 1.0e-9
# Trust-region radius, needs to be actively modified

b = -r0.weighted()
assert b.size == W.shape[1]
UMat, dArr, VHMat = svd(H.T)
assert dArr.ndim == 1
bHat = np.dot( UMat.conj().T, np.dot(W.conj(),b) )

sHat = lambda mu: bHat*dArr/(dArr**2 + mu )

# Important: I must ensure that mu is a positive scalar
phi = lambda mu: norm( sHat(mu), 2 )**2 - delta**2
phiPrime = lambda mu: -2.* np.sum(  sHat(mu)**2/(dArr**2+mu)   )

# Need to run a modified Newton's search on phi(mu) = 0 
mu = 1.
tol = 1.0e-14
nIter = 15
muArr = np.zeros(nIter)
print('beginning Newton search for mu...........')
for k in range(nIter):
    muArr[k] = mu
    if np.abs(phi(mu)) <= tol: 
        print('Newton iterations for mu have converged for k =',k)
        break
    mu = mu - norm( sHat(mu),2)/delta*phi(mu)/phiPrime(mu)
    if mu < 0.: 
        print('mu has gone negative, setting it to zero..')
        mu = 0.
    
        

In [ ]:
mu

In [ ]:
sHat(mu).shape, VHMat.shape

## FFT and Convolution of spectral coefficients for 1-D arrays


In [ ]:
import scipy.signal as sig
def fftMan(y,x): # Manual DFT
    if y[-1]==y[0]: _y = y[:-1]; _x=x[:-1]
    else: _y = y; _x = x
    global alpha
    N = _y.size//2
    coeffs = np.zeros(2*N+1, dtype=np.complex)
    for k in range(-N,N+1):
        coeffs[N+k] = np.dot( _y,np.exp(-1.j*k*alpha*_x) )
    
    coeffs = coeffs/_y.size
    
    if norm(np.imag(y)) == 0.: return np.real(coeffs)
    return coeffs
    
def ifftMan(y,x): # Manual inverseDFT
    assert y.size//2 != 0.
    _y = y
        
    global alpha
    N = _y.size//2
    phys = np.zeros(2*N,dtype=np.complex)
    for k in range(-N,N+1):
        phys += _y[N+k]*np.exp(1.j*k*alpha*x) 
    
    return np.real(phys)

def convMan(y1,y2): # convolution: pad zeros on both sides, do convolution, return middle part
    # I'm supposing the convolution is on the spectral coeffs
    N = y1.size//2
    zArr = np.zeros(N,dtype=np.complex)
    
    res = np.zeros(y1.size+2*N, dtype=np.complex)
    
    for k1 in range(-N,N+1):
        for k2 in range(-N,N+1):
            res[2*N+k1+k2] += y1[N+k1] * y2[N+k2]
    
    return res[N:N+y1.size]

def npfft(x):
    assert x.size//2 != 0., "Numpy's fft only works well on [0,2*pi), i.e., when 2*pi is not included"
    c = np.fft.fftshift(np.fft.fft(x))/x.size
    c = np.append(c,np.conj(c[0]))
    if norm(np.imag(c),2)/c.size<= 1.0e-14: return np.real(c)
    return c

def npifft(x):
    assert x.size//2 != 0., "Coefficient vector must have modes -N through +N, including +N"
    c = np.fft.ifft(np.fft.ifftshift(  x[:-1]*(x.size-1)    ))
    assert norm(np.imag(c),2)/c.size<=1.0e-14, "I only use real fields, so ifft should not have any imaginary parts"
    return np.real(c)

def npconvolve(x,y):
    return np.convolve(x,y,mode='same')

def isZero(x):
    return not (np.abs(x)>1.0e-14).any()



In [ ]:
# Testing convolution theorem: convolve(F(y1),F(y2)) = F(y1*y2)
N = 5; A = 2.; alpha = 3.; M = -1
x = (np.arange(-N,N)+N)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)

c1 = npfft(y1)
c2 = npfft(y2)

#print(npfft(y1))
print("Does npfft comply with convolution theorem, convolution in spectral?:", isZero(npconvolve(npfft(y1),npfft(y2))-npfft(y1*y2))                  )
print("Does npfft match fftMan?:", isZero(fftMan(y1,x)-npfft(y1))                  )
print("Is ifftMan consistent with fftMan?:", isZero( ifftMan(fftMan(y1,x),x)-y1  ) )


## Ensure npfft and npifft (both custom functions) are consistent with each other


In [ ]:
# First, showing that numpy's routines are self-consistent
print("Are numpy's fft and ifft self-consistent?:",isZero(np.fft.ifft(np.fft.fft(y1)) - y1))

# Now, let's see compare what ifftshift to two things:
#   1. ifftshift(fftshift(np.fft.fft))
#   2. ifftshift(npfft)   - This one is the function I wrote where I append
#       the coefficient for mode +N as the complex conjugate of mode -N:
c1 = npfft(y1)
cp1 = np.fft.fftshift(np.fft.fft(y1))
if False:
    print("ifftshift on custom function:",np.fft.ifftshift(c1))
    print("ifft of custom function:", np.real(np.fft.ifft(np.fft.ifftshift(c1*(c1.size)))))
    print("ifft of numpy's fft:", np.real(np.fft.ifft(np.fft.ifftshift(cp1))))
    print("Original function y1:",y1)
    print("ifft of custom function, ignoring +N mode:", np.real(np.fft.ifft(np.fft.ifftshift(c1[:-1]*(c1.size-1)))))

    print("Are npfft and npifft consistent with each other?", isZero(npifft(npfft(y1))-y1) and isZero( npifft(npfft(y2))-y2 )   )

In [ ]:
# Checking if sp.fftconvolve gives the same results
N = 50 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("Comparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


N = 500 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("\n\nComparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("Do np.convolve and sp.fftconvolve give the same result?:", 
      isZero(np.convolve(c1,c2,mode='same')- sig.fftconvolve(c1,c2,mode='same'))  )
print("Do np.convolve and custom convolve give the same result?:", 
      isZero( np.convolve(c1,c2,mode='same')- npfft(npifft(c1)*npifft(c2)) )  )
print("Do np.convolve and scipy.signal.convolve give the same result?:", 
      isZero( np.convolve(c1,c2,mode='same')- sig.convolve(c1,c2,mode='same') )  )


print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


N = 25000 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("\n\nComparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


## Figuring out how np.fft.fftn and np.fft.ifftn work

In [ ]:
Nx = 3; Nz = 3
A = 2.; 
alpha = 3.; L = 2; beta= 1.5; M= 1
#y = vF.y; 
y = np.array([-1.,-0.3,0.3,1.])
y=y.reshape((1,1,y.size))
x = (np.arange(-Nx,Nx)+Nx)/Nx*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
z = (np.arange(-Nz,Nz)+Nz)/Nz*np.pi/beta
x = x.reshape((x.size,1,1))
z = z.reshape((1,z.size,1))

f1 = A*np.cos(L*alpha*x + M*beta*z)*(1.-y**2)
f2 = A*np.cos(alpha*x + 2*beta*z)*(1.-y**4)
print(f1.shape)

In [ ]:
# The following code works
cp1 = np.fft.fftn(f1,axes=[0,1])/f1.size*4
c1 = np.fft.fftshift(cp1, axes=[0,1])
c1 = c1[1:,1:]

#np.real(np.fft.fftn(f1,axes=[0,1]))/f1.size
np.real(c1[:,:,1]/(1.-0.3*0.3))

In [ ]:
s1 = np.zeros((7,7,4), dtype=np.complex)
s1[1,2] = A/2.
s1[5,4] = A/2.
s1[:] *= 1.-y**2

s1 = s1[:-1,:-1]

%timeit p1 = np.fft.ifftn(  np.fft.ifftshift(s1,axes=[0,1]), axes=[0,1] )*(s1.shape[0]*s1.shape[1])

#c1[:,:,1], s1[1:,1:,1]
isZero(p1- f1)
norm((p1-f1).reshape(p1.size),2)

## Convolution and FFT for 2-d arrays, Fourier-spectral on axis 0

This is the intermediate step before I finally move to my flowField objects

In [ ]:
def convMan(ff1,ff2):
    """Returns convolution of flowField instances ff1 and ff2 along x and z as a 3d numpy array
    The assumption here is that they're both in spectral. 
    Convolution is computed by first doing an ifft of both arrays along axes given by argument axes,
        the arrays in physical space are multiplied, and the result is then fft'd
    I use numpy's fft, which is a bit unintuitive. I have to pad ff1 and ff2 before the ifft"""
    assert (ff1.nd==1) and (ff2.nd==1)
    # Padding with an extra wavenumber on both dimensions, this will be discarded later
    _f1 = ff1.slice(L=ff1.nx//2+1, M=ff1.nz//2+1)
    _f2 = ff2.slice(L=ff2.nx//2+1, M=ff2.nz//2+1)
    
    # Discarding the last positive modes, because numpy's fft doesn't like it if it was in there
    _f1 = _f1.view4d().copyArray()
    _f2 = _f2.view4d().copyArray()
    _f1 = _f1[0,:-1,:-1,0]
    _f2 = _f2[0,:-1,:-1,0]
    
    # Arranging modes in the order that numpy's fft likes, obtaining array in physical space
    ph1 = np.fft.ifftn(  np.fft.ifftshift(_f1, axes=[0,1]), axes=[0,1]  )*(_f1.shape[0]*_f1.shape[1])
    ph2 = np.fft.ifftn(  np.fft.ifftshift(_f2, axes=[0,1]), axes=[0,1]  )*(_f1.shape[0]*_f1.shape[1])
    
    # Convolution as product in physical space
    prod = ph1*ph2
    
    # Convolution by fft'ing product, and then shifting to the ordering I like
    conv = np.fft.fftshift(  np.fft.fftn(prod,axes=[0,1]), axes=[0,1] )/(_f1.shape[0]*_f1.shape[1])
    
    # Removing the last negative mode, which I only padded in.
    conv = conv[1:,1:]
    
    return conv      

In [ ]:
#vF = mapData2ff(eps=0.01, g=0.8, Re=100., theta=0)[0]
vF = vf.copy()
#vF[:,:,:,1:] = 0.
u = vF.getScalar(); v=vF.getScalar(nd=1); w=vF.getScalar(nd=2)

%timeit vF.convNL().getScalar().view4d().copyArray()
convClassArr = vF.convNL().getScalar().view4d().copyArray()
convClassArr = convClassArr[0,:,:,0]

%timeit convMan(u,u.ddx())
convCustom = convMan(u,u.ddx()) + convMan(v,u.ddy()) + convMan(w,u.ddz())       
    
diff = convClassArr - convCustom
print(norm(convCustom.reshape(convCustom.size)), norm(diff.reshape(diff.size)))

In [ ]:
#(vf.convNL(fft=False) - vf.convNL()).norm()
start = time.time()
conv1= vf.convNL()
stop = time.time()
print(stop-start)
conv2 = vf.convNL(fft=False)
print(time.time()-stop)


In [ ]:
Y = np.append(y1.reshape((y1.size,1)),y2.reshape((y2.size,1)),axis=1)
C = npfft2d(Y)

isZero(npconvolve2d(C,C) - npfft2d(Y*Y))

In [ ]:
arr = np.array([0,1,2,3,-3,-2,-1]).reshape((7,1))
arr = arr*arr.reshape((1,7))
print(arr)
np.fft.fftshift(arr, axes=[0,1])

## Using scipy.signal.convolve2d (for each wall-normal node) to do the convolution


In [ ]:
#vF = mapData2ff(eps=0.02,g=0.8,Re=100.,theta=0)[0]
vF = vf.copy()
vF[:,:,:,1:] = 0.
u = vF.getScalar()
uArr = u.view4d().copyArray(); duArr = u.ddx().view4d().copyArray()

uArr = uArr[0,:,:,0]; duArr = duArr[0,:,:,0]

%timeit vF.convNL().getScalar().view4d().copyArray()
convClassArr = vF.convNL().getScalar().view4d().copyArray()
print(convClassArr.shape)
convClassArr = convClassArr[0,:,:,0]

        
convConvolve = uArr.copy(); 
%timeit [sig.convolve2d(uArr[:,:,k],duArr[:,:,k],mode='same') for k in range(vF.N)] 

%timeit sig.convolve2d(uArr[:,:,5],duArr[:,:,5])

for k in range(vF.N): 
    convConvolve[:,:,k] = sig.convolve2d(uArr[:,:,k],duArr[:,:,k],mode='same') 
    
diff = convClassArr - convConvolve
print(norm(convConvolve.reshape(convConvolve.size)), norm(diff.reshape(diff.size)))



In [ ]:
%timeit vf.slice(L=vf.nx//2+1,M=vf.nz//2+1)

In [ ]:
f = h5py.File("eq1_pressure.h5",'r')


In [ ]:
p = np.array(f['data']['u'])

In [ ]:
for name in f['geom']: print(name)

In [ ]:
vF = h52ff('eq1.h5')
pF = h52ff('eq1_pressure.h5',pres=True)

In [ ]:
x0 = vF.appendField(pF)
res = residual(x0)
res.norm()*Lx*Lz

In [ ]:
norm(res.view1d(),2)/Lx/Lz/2.

In [ ]:
ffArr = x0.copyArray().reshape((x0.size//N,N))
for k in range(x0.size//N):
    ffArr[k] = chebcoeffs(ffArr[k])
    #ffArr[k,-1] = 0.
    ffArr[k] = chebcoll_vec(ffArr[k])

x=x0.copy()
x.view1d()[:] = ffArr.reshape(ffArr.size)

In [ ]:
vFluct = vF.copy()
vFluct[0,23,23,0] -= vFluct.y
vFluct.norm()*Lx*Lz

In [ ]:
vFluctPhys = np.fft.ifftn(  np.fft.ifftshift(vFluct.view4d().copyArray()[:,:-1,:-1], axes=[1,2]), axes=[1,2] )*46.*46.


In [ ]:
norm(vFluctPhys.reshape(vFluctPhys.size), 2)/Lx/Lz

In [ ]:
Lx = 2.*np.pi/vFluct.flowDict['alpha']
Lz = 2.*np.pi/vFluct.flowDict['beta']

In [ ]:
norm(vFluct.view1d(),2)/Lx/Lz/2.

In [ ]:
ff = vF.slice(L=0,M=0,N=3,nd=[0])
w = clencurt(3)

In [ ]:
w

In [ ]:
ff.shape

In [ ]:
ff[:] = np.array([1.,1.,1.])
ff.norm()

In [ ]:
norm(vFluct.view1d(),2)/Lx/Lz/2.